# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126341e+02     1.515004e+00
 * time: 0.5650050640106201
     1     1.069059e+01     9.165925e-01
 * time: 1.862800121307373
     2    -1.160164e+01     1.044819e+00
 * time: 1.9748001098632812
     3    -3.406116e+01     8.572100e-01
 * time: 2.0704360008239746
     4    -4.754233e+01     6.863284e-01
 * time: 2.1729049682617188
     5    -5.691714e+01     2.589551e-01
 * time: 2.295480966567993
     6    -5.973902e+01     1.969510e-01
 * time: 2.360038995742798
     7    -6.087405e+01     6.644057e-02
 * time: 2.4248759746551514
     8    -6.135081e+01     3.544961e-02
 * time: 2.488456964492798
     9    -6.160676e+01     3.839319e-02
 * time: 2.5533270835876465
    10    -6.177246e+01     3.195089e-02
 * time: 2.634108066558838
    11    -6.191638e+01     2.060801e-02
 * time: 2.697000026702881
    12    -6.202226e+01     1.900799e-02
 * time: 2.7597241401672363
    13    -6.207410e+01     1.534422e-02
 * time: 2.8225371837615967

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671067
    AtomicLocal         -18.8557686
    AtomicNonlocal      14.8522655
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666460484